## Name: Krina Bharatbhai Patel
## Net Id : vj3004

## Name: Fnu,Sharmila Bolikoppa Palakshappa
## Net Id : bm6575

## Name: Vatsal Sudhir Pathak
## Net Id : pv2556

## Group Id: 7
## Assignment 2

## Q4 [1pts]: Use the first 10 features in Q1 that gave the best training performance and repeat Q2 using those 10 dimensions (columns). Compare the test error performance you got to Q2. 




In [21]:
import numpy as np
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
import pandas as pd
from numpy.linalg import multi_dot

df_train = read_csv('optdigits_69_N200.tra',header=None)
df_test = read_csv('optdigits_69.tes',header=None)

In [22]:
# computing variance for each columns for each class
# if variance is below or equal to 0(var_threshold) for any column for any class, that column will be dropped

var_threshold = 0
df_var = df_train.groupby(by=64).var()
s_var_threshold = (df_var > var_threshold).product()
columns_to_drop = s_var_threshold[s_var_threshold == 0].index
print(columns_to_drop)

Int64Index([0, 1, 6, 7, 8, 14, 15, 16, 21, 22, 23, 24, 31, 32, 39, 40, 47, 48,
            56],
           dtype='int64')


In [23]:
# drop columns with variance below or equal to  threshold
df_train = df_train.drop(columns = columns_to_drop)
df_test = df_test.drop(columns = columns_to_drop)

In [24]:
df_train.head()

,2,3,4,5,9,10,11,12,13,17,...,54,55,57,58,59,60,61,62,63,64
0,5,14,4,0,0,13,8,0,0,3,...,12,0,0,4,12,14,7,0,0,6
1,0,10,12,0,0,9,14,4,0,0,...,14,0,0,1,11,14,12,1,0,6
2,2,14,10,0,1,14,12,0,0,5,...,15,0,0,1,12,14,12,4,0,6
3,2,13,5,0,0,11,15,0,0,1,...,6,0,0,4,15,12,9,1,0,6
4,2,13,6,0,0,10,14,6,0,0,...,13,0,0,2,12,14,12,4,0,6


# Find statistics (mean, covariance and probability) for each label (6,9)

In [25]:
m = df_train.groupby(by=64).mean()
s = df_train.groupby(by=64).std()
count = df_train.groupby(by=64).count()
prob = count / count.sum()

## Linear Discriminant Function

In [26]:
# g_i(x) for i = 1, 2 i.e, two discirminant function for class 6 and 9

def discriminant_predict(x):
    g = -0.5 * np.log(2*np.pi) - np.log(s) - ((x - m) **2)/(2*(s**2)) + np.log(prob)
    return g.idxmax()

## Predict Function

In [27]:
df_train_no_label = df_train.drop(columns=[64])

results = []
for i in range(df_train_no_label.shape[0]):
    x = df_train_no_label.iloc[i]
    res = discriminant_predict(x)
    results.append(res)

In [28]:
df_results = pd.DataFrame(results)
df_results

,2,3,4,5,9,10,11,12,13,17,...,53,54,55,57,58,59,60,61,62,63
0,9,9,6,6,6,9,9,6,6,9,...,6,6,9,6,6,6,6,9,9,9
1,6,6,9,6,6,6,6,6,6,6,...,6,6,9,6,6,6,6,6,9,9
2,6,9,9,6,6,9,6,6,6,9,...,6,6,9,6,6,6,6,6,9,9
3,6,6,6,6,6,9,6,6,6,6,...,9,9,9,6,6,9,9,9,9,9
4,6,6,6,6,6,9,6,6,6,6,...,6,6,9,6,6,6,6,6,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6,6,9,9,6,6,6,9,9,6,...,6,9,9,6,6,9,6,9,9,9
196,6,9,9,9,6,6,6,6,9,6,...,9,9,9,6,6,9,6,9,9,9
197,6,9,9,9,6,6,6,9,9,6,...,9,9,9,6,6,9,9,9,9,9
198,9,9,9,9,9,9,9,9,9,9,...,6,6,9,6,9,9,6,6,9,9


## Find train and test errors

In [29]:
s1 = df_results.ne(df_train[64], axis=0).mean().sort_values()*100

In [30]:
df_test_no_label = df_test.drop(columns=[64])

results_test = []
for i in range(df_test_no_label.shape[0]):
    x_test = df_test_no_label.iloc[i]
    res_test = discriminant_predict(x_test)
    results_test.append(res_test)

In [31]:
df_results_test = pd.DataFrame(results_test)
s2=df_results_test.ne(df_test[64], axis=0).mean().sort_values()*100


In [32]:
#displaying training and testing error next to eaxh other
df_final = pd.concat([s1, s2], axis=1)
df_final.columns=["Train_error","Test_error"]    
df_final.sort_values("Train_error")

,Train_error,Test_error
42,0.5,1.108033
29,2.0,8.033241
13,8.0,7.202216
34,9.5,8.310249
30,9.5,13.296399
43,9.5,9.141274
28,15.5,15.789474
54,17.5,26.592798
20,18.5,14.958449
51,20.0,16.620499


## 10 features in Question-1 that gave the best training performance 

In [33]:
best_features = [42, 29, 13, 34, 30, 43, 28, 54, 20, 51, 64]
df_train_class = df_train[best_features]
df_test_class = df_test[best_features]
df_test_class

,42,29,13,34,30,43,28,54,20,51,64
0,13,0,0,15,0,16,0,8,3,16,6
1,15,0,0,7,0,16,2,16,7,14,6
2,16,0,2,16,0,16,1,12,3,16,6
3,13,0,0,15,0,14,5,9,2,16,6
4,16,0,0,16,0,16,0,14,0,16,6
...,...,...,...,...,...,...,...,...,...,...,...
356,0,16,16,0,6,0,15,7,6,9,9
357,0,15,16,0,4,0,16,0,12,8,9
358,0,16,16,0,5,0,13,6,16,6,9
359,0,16,16,0,5,0,16,4,3,7,9


## Find statistics (mean, covariance matrix, inverse covariance matrix and probability) for each label (6,9)

In [34]:
grouped = df_train_class.groupby(by=64)
stat_list = []
for name, group in grouped:
    df = group.drop(columns=[64])
    m = np.mean(df)
    s = np.cov(df.T)
    s_inv = np.linalg.inv(s)
    prob = df.shape[0] / df_train.shape[0]
    stat_list.append([name, m, s, s_inv, prob])

In [35]:
df_stat = pd.DataFrame(stat_list, columns=['label', 'mean', 'cov', 'cov_inv', 'prob'])
df_stat = df_stat.set_index('label')
df_stat

,mean,cov,cov_inv,prob
label,,,,
6,42 14.98 29 0.92 13 0.79 34 14.7...,"[[2.282424242424246, 0.3115151515151519, -0.09...","[[0.5227402595785623, -0.06359626185704549, -0...",0.5
9,42 0.43 29 14.76 13 10.58 34 3.5...,"[[2.7122222222222216, -0.4109090909090913, 1.2...","[[0.42096502984188877, 0.060314168244284774, -...",0.5


## Multivariate Discriminant Function

In [36]:
def discriminant_func(x, mean, cov, cov_inv, prob):    
    wi = -0.5 * cov_inv
    wj = np.dot(cov_inv, mean)
    wk = -0.5 * np.linalg.multi_dot([mean.T, cov_inv, mean]) - 0.5 * np.log(np.linalg.det(cov)) + np.log(prob)
    g = np.linalg.multi_dot([x.T, wi, x]) + np.dot(wj.T, x) + wk 
    return g.squeeze().astype('float')

## Multivariate Predict Function

In [37]:
def multivariate_predict(df, df_stat):
    df_no_label = df.drop(columns=[64])

    results = []
    for i in range(df_no_label.shape[0]):
        x = np.asarray(df_no_label.iloc[i])
        x = x.reshape(1,-1).T
        discriminant_values = df_stat.apply(lambda row:
                            discriminant_func(x, row["mean"], row["cov"], row["cov_inv"], row["prob"]), axis=1)

        discriminant_values = discriminant_values.astype('float')
        res = discriminant_values.idxmax()
        results.append(res)

    df_results = pd.DataFrame(results, columns=['predicted'])
    return df_results


## Find train and test errors

In [38]:
df_results_train = multivariate_predict(df_train_class, df_stat)
df_results_test = multivariate_predict(df_test_class, df_stat)

In [39]:
e = df_results_train.ne(df_train_class[64], axis=0).mean().sort_values()*100
print("Train error: %.4f %%"%(e.values[0]))

Train error: 0.0000 %


## Train error: 0.0000 %

In [40]:
e2 = df_results_test.ne(df_test_class[64], axis=0).mean().sort_values()*100
print("Test error: %.4f %%"%(e2.values[0]))

Test error: 2.4931 %


## Test error: 2.4931 %

## Comparing the test error performance from Question-2

| Features | Train Error | Test Error |
| --- | --- | --- |
| 64 | 0.0000 % | 1.3850 % |
| 10 | 0.0000 % | 2.4931 % |